In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from scipy import stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.formula.api as smf

from statsmodels.stats.multicomp import pairwise_tukeyhsd

import re
import nltk

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kucza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
def ListNoDups(mylist):
    mylist = list(dict.fromkeys(mylist))
    return mylist

class CarData:
    
    missing = -1
    duplicates = -1
    
    price_outliers = -1
    mileage_outliers = -1
    year_outliers = -1
    total_discard = -1
    
    corpus = []
    def __init__(self, path, price_outlier_mt = 200000, mileage_outlier_mt = 400000,
                year_outlier_lt = 1995, engine_outlier_mt = 4000, engine_outlier_lt = 750,
                dependent_variable = 'price'):
        
        #define outliers values
        self.price_outlier_mt = price_outlier_mt
        self.mileage_outlier_mt = mileage_outlier_mt
        self.year_outlier_lt =  year_outlier_lt
        self.engine_outlier_mt = engine_outlier_mt
        self.engine_outlier_lt = engine_outlier_lt
        self.dependent_variable = dependent_variable

        
        
        #defince variable data types
        self.numeric_variables = ['price', 'mileage_km', 'engine_cm3', 'year']
        self.categorical_variables = ['engine_type', 'city', 'province']
        
        #read all .csv files from the directory
        self.data = pd.concat(map(pd.read_csv, glob.glob(os.path.join(path, "*.csv"))), sort=False)
        
        #drop the duplicates and save the number of duplicates - many duplicates due to data gathering method
        self.duplicates = len(self.data) - len(self.data.drop_duplicates())
        self.data.drop_duplicates(inplace = True)
        
        #rename columns
        self.data.columns = ['title', 'price', 'sub_title', 'mileage_km', 'year', 'engine_cm3',
                'engine_type', 'city', 'province', 'negotiable']
        
        #drop NaNs and save the number of rows dropped to the missing varaible
        self.missing = self.data['engine_type'].isna().sum()
        self.data.dropna(subset = ['engine_type'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['city'].isna().sum()
        self.data.dropna(subset = ['city'], axis = 'index', inplace = True)
        
        self.missing = self.missing + self.data['engine_cm3'].isna().sum()
        self.data.dropna(subset = ['engine_cm3'], axis = 'index', inplace = True)
        
        #clean up the columns
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(",", ""))
        self.data['price'] = self.data['price'].apply(lambda x: x.replace(" ", "")).astype('int')
        
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace("km", ""))
        self.data['mileage_km'] = self.data['mileage_km'].apply(lambda x: x.replace(" ", "")).astype('float')
        
        self.data['engine_cm3'] = self.data['engine_cm3'].astype('str')
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace('cm3', ''))
        self.data['engine_cm3'] = self.data['engine_cm3'].apply(lambda x: x.replace(' ','')).astype('int')
        
        self.data['province'] = self.data['province'].astype('str')
        self.data['province'] = self.data['province'].apply(lambda x: x.replace('(',''))
        self.data['province'] = self.data['province'].apply(lambda x: x.replace(')',''))
        
        self.data['sub_title'] = self.data['sub_title'].astype('str') #may change that in the future - possible info loss due to lowercase
        
        
        self.data['title'] = self.data['title'].astype('str') #may change that in the future - possible info loss due to lowercase
        
    
        #Add ID column
        self.data.insert(loc = 0, column = 'ID', value = range(1, len(self.data)+1))

        #discard outliers and calculate the numbers
        self.total_discard = len(self.data) - len(self.data[(self.data['price'] <= self.price_outlier_mt) &
                                                        (self.data['mileage_km'] <= self.mileage_outlier_mt) &
                                                        (self.data['year'] >= self.year_outlier_lt) &
                                                        (self.data['engine_cm3'] <= self.engine_outlier_mt) &
                                                        (self.data['engine_cm3'] >= self.engine_outlier_lt)])        
        
        self.price_outliers = len(self.data[self.data['price'] > price_outlier_mt])
        self.data = self.data[self.data['price'] <= price_outlier_mt]
        
        self.mileage_outliers = len(self.data[self.data['mileage_km'] > mileage_outlier_mt])
        self.data = self.data[self.data['mileage_km'] <= mileage_outlier_mt]
        
        self.year_outliers = len(self.data[self.data['year'] < year_outlier_lt])
        self.data = self.data[self.data['year'] >= year_outlier_lt]
        
        self.engine_outliers = len(self.data[(self.data['engine_cm3'] > engine_outlier_mt) |
                                            (self.data['engine_cm3'] < engine_outlier_lt)])
        self.data = self.data[(self.data['engine_cm3'] <= engine_outlier_mt) & 
                             (self.data['engine_cm3'] >= engine_outlier_lt)]
        
        #NLP
        self.data['concat_title_subtitle'] = self.data['title'] + ' ' + self.data['sub_title']
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.lower())
        
        #replace problematic cases for NLP
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('+','plus'))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('(',''))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(')',''))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('**',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('*',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(']',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('[',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace("/"," "))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace("\\"," "))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(',',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('?',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('.',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('!',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('_',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('-',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('|',' '))
        self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace('#',' '))
        
    def describe(self):
        #descriptive statistice
        desc_stats = round(pd.DataFrame(
                        data = self.data[['price', 'mileage_km', 'year', 'engine_cm3']].describe(),
                        columns = self.data[['price', 'mileage_km', 'year', 'engine_cm3']].columns),2)
        return desc_stats
    
    def outliers(self):
        #baisc data about outliers discarded during preprocessing
        print('Offers with price greater than '+str(self.price_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.price_outliers))
        print('')
        print('Offers with mileage greater than '+str(self.mileage_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.mileage_outliers))
        print('')
        print('Offers with year lower than '+str(self.year_outlier_lt)+' have been discarded')
        print('The number of such offers = '+str(self.year_outliers))
        print('')
        print('Offers with engine_cm3 greater than '+str(self.engine_outlier_lt)+
              ' and lower than '+str(self.engine_outlier_mt)+' have been discarded')
        print('The number of such offers = '+str(self.engine_outliers))
        print('')
        print('Total number of discarded offers = '+str(self.total_discard)
              +'('+str(round(self.total_discard/len(self.data)*100,2))+'%)'
              +' - may be different to the sum of above due to overlap')
        
    def scatter_nox(self, var = 'all', figsize_1 = 7, figsize_2 = 5):
        #prints scatter plots with no x axis - a dummy sequence as x axis
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        if var == 'all':
            plt.rcParams["figure.figsize"] = (figsize_1,figsize_2)
            plt.scatter(y = self.data['mileage_km'], x = range(1, len(self.data)+1), s=1)
            plt.title('mileage_km')
            plt.show()

            plt.scatter(y = self.data['price'], x = range(1, len(self.data)+1), s=1)
            plt.title('price')
            plt.show()

            plt.scatter(y = self.data['year'], x = range(1, len(self.data)+1), s=1)
            plt.title('year')
            plt.show()

            plt.scatter(y = self.data['engine_cm3'], x = range(1, len(self.data)+1), s=1)
            plt.title('engine_cm3')
            plt.show()
        else:
            plt.rcParams["figure.figsize"] = (figsize_1,figsize_2)
            plt.scatter(y = self.data[var], x = range(1, len(self.data)+1), s=1)
            plt.title(var)
            plt.show()
            
    def scatter(self, var = 'all'):
        #prints scatter plots for numerical variables
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        list_comb = []
        if var == 'all':
            for variable1 in enumerate(self.numeric_variables):
                for variable2 in enumerate(self.numeric_variables):
                    if variable1 != variable2 and variable1[1]+variable2[1] not in list_comb and variable2[1]+variable1[1] not in list_comb:
                        plt.scatter(y = self.data[variable1[1]], x = self.data[variable2[1]], s=1)
                        plt.title("Correlation between "+variable1[1]+' and '+variable2[1])
                        plt.ylabel(variable1[1])
                        plt.xlabel(variable2[1])
                        plt.show()
                        list_comb.append(variable1[1]+variable2[1])
        #else: - TO DO
    
    def hist(self, var = 'all', bins = 50):
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        #prints histograms for numeric variables
        if var == 'all':
            for variable in enumerate(self.numeric_variables):
                plt.hist(x = self.data[variable[1]], bins = bins)
                plt.title(variable[1])
                plt.show()
     
    def price_cat_vars(self):
        for variable in enumerate(self.categorical_variables):
            # shows desrptive statistics of categorical variables
            print(x.data.groupby(x.data[variable[1]])['price'].describe())
            #the variables need further preprocessing
            
    def add_dummies(self, categorical_list, columns_to_check, delete_from_strings = 'yes'):
        #adds dummmies from cat_list, checks in every column of columns
        for column in enumerate(columns_to_check):
            for category in enumerate(categorical_list):
                col_name = column[1] + '_' + category[1]
                self.data[col_name] = self.data[column[1]].str.contains(cat[1]).astype('int')
                
                #append newly craeted varaibles to categorical variables
                if self.data[col_name].sum() > 0:
                    self.categorical_variables.append(col_name)
                else:
                    self.data.drop(columns = [col_name], inplace = True)
                
                #delete the string from the column
                if delete_from_strings == 'yes':
                    self.data[column] = self.data[column].apply(lambda x: x.replace(category[1], ''))
                
        return self
    
    def add_dummies2(self, categorical_list, delete_from_strings = 'yes'):
        #adds dummmies from cat_list, checks in every column of columns
        for category in enumerate(categorical_list):
            col_name = category[1]
            self.data[col_name] = self.data['concat_title_subtitle'].str.contains(category[1]).astype('int')
                
                #append newly craeted varaibles to categorical variables
            if self.data[col_name].sum() > 0:
                self.categorical_variables.append(col_name)
            else:
                self.data.drop(columns = [col_name], inplace = True)
                
                #delete the string from the column
            if delete_from_strings == 'yes':
                self.data['concat_title_subtitle'] = self.data['concat_title_subtitle'].apply(lambda x: x.replace(category[1], ''))
                
        return self
    """
    def ind_test(self, var, alpha = 0.05):
        if alpha > 1 or alpha < 0:
            print('Incorrect alpha value. Select a value from <0;1>.')
            
        if var != 'all' and var not in self.data.columns:
            print('Variable not found in the dataset')
        pivot = round(self.data.pivot_table(values = 'price', index = var, aggfunc = ['count', 'mean']),2)
        pivot.columns = ['count', 'mean']
        
        mean_price = self.data['price'].mean()
        
        pivot['sm'] = pivot['mean']/((pivot['count'])**(1/2))
        
        pivot['t'] = (pivot['mean']-mean_price)/pivot['sm']
        pivot['df'] = pivot['count']-1

        #calculate p-value
        pivot['t_border'] = stats.t.ppf(1-alpha/2, pivot['df'])
        
        #implementation here is not 100% mathematically correct
        return pivot
    """
    def anova(self, var = 'all', alpha = 0.05):
        if var == 'all':
            for variable in enumerate(self.categorical_variables):
                anova_data = self.data[[variable[1], 'price']].reset_index().copy()
                anova_data.columns = ['index', variable[1], 'price']
                equation_string = 'price ~ '+str(variable[1])
                model = ols(equation_string, data=anova_data).fit()
                anova_table = sm.stats.anova_lm(model, typ=2)
                print(anova_table)
                print()
                
                #pairwise comparisons
                pairwise_comparison = pairwise_tukeyhsd(endog = anova_data['price'],
                                                        groups = anova_data[variable[1]],
                                                        alpha = alpha)
                print(pairwise_comparison)
                print()
        #else:
            #TO DO
            #also TO DO check Anova assumptions
    def make_corpus(self):
        #creates a corpus out of title and subtitle column
        for i in range(0, len(self.data)):
            #string = re.sub('[^a-zA-Z]', ' ', self.data.reset_index().loc[i]['concat_title_subtitle'])
            string = self.data.reset_index().loc[i]['concat_title_subtitle']
            string = string.split()
            self.corpus = self.corpus + string
        self.corpus = ListNoDups(self.corpus)
        
        #with open("corpus.txt", "w") as output:
         #   output.write(str(self.corpus))
    
        return self.corpus
    
    def analyse_variables(self, list_of_variables, discard = 0.01):
    # independence tests for a list of variable e.g. corpus
        final_df = pd.DataFrame(columns = ['variable', 'mean_1', 'mean_0', 'count_1', 'count_0'])

        for variable in enumerate(list_of_variables):
            #debug
            print(str(variable)+' done')
            
            self.data[variable[1]] = self.data['concat_title_subtitle'].str.contains(variable[1]).astype('int')

            mean_1 = self.data.loc[self.data[variable[1]] == 1][self.dependent_variable].mean()
            mean_0 = self.data.loc[self.data[variable[1]] == 0][self.dependent_variable].mean()

            count_1 = len(self.data.loc[self.data[variable[1]] == 1])
            count_0 = len(self.data.loc[self.data[variable[1]] == 0])        
            
            if count_1 >= discard * len(self.data) and count_0 >= discard * len(self.data):            
                dict_to_append = {
                    'variable' : variable[1],
                    'mean_1' : mean_1,
                    'mean_0' : mean_0,
                    'count_1' : count_1,
                    'count_0' : count_0
                }

                final_df = final_df.append(dict_to_append, ignore_index = True)

            self.data.drop(columns = [variable[1]], inplace = True)

            if variable[0] % 1000 == 0:
                print(str(variable[0])+'/'+str(len(list_of_variables)))
            
            
        
        
        final_df['mean_diff'] = abs(final_df['mean_1'] - final_df['mean_0'])
        final_df = final_df.sort_values(by = 'mean_diff', ascending = False).reset_index()
        
        return final_df
            

#cv = CountVectorizer(max_features = 1000)
#X = cv.fit_transform(corpus).toarray()

In [43]:
x = CarData('data/', dependent_variable = 'price')

In [44]:
corpus = x.make_corpus()

In [4]:
with open("corpus.txt", "r") as file:
    corpus = eval(file.readline())

SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [ ]:
results = x.analyse_variables(list_of_variables = corpus)

(0, 'mazda') done
0/14385
(1, 'cx') done
(2, '3') done
(3, '150km') done
(4, '6at') done
(5, '4x4') done
(6, 'skypassion') done
(7, 'plus') done
(8, 'biala') done
(9, 'skóra') done
(10, 'safety') done
(11, 'navi') done
(12, 'peugeot') done
(13, 'partner') done
(14, 'opłacony~dokumentacja') done
(15, 'przebiegu') done
(16, 'kia') done
(17, 'sportage') done
(18, 'opłacona~napęd') done
(19, 'škoda') done
(20, 'roomster') done
(21, 'opłacona~serwis') done
(22, 'aso~klimatronic~grzane') done
(23, 'siedzenia') done
(24, 'opel') done
(25, 'combo') done
(26, 'klimatyzacja~przebieg~faktura~polski') done
(27, 'salon') done
(28, 'mitsubishi') done
(29, 'lancer') done
(30, '1') done
(31, '8') done
(32, 'polska') done
(33, '24000') done
(34, 'bmw') done
(35, 'seria') done
(36, '320d') done
(37, '5') done
(38, 'e60') done
(39, '535d') done
(40, '272km') done
(41, '2006r') done
(42, 'head') done
(43, 'up') done
(44, 'start') done
(45, 'stop') done
(46, 'xenony') done
(47, 'bogate') done
(48, 'wyposaż

(406, 'rx') done
(407, '350') done
(408, 'ga') done
(409, 'właścicielka') done
(410, 'a6') done
(411, 'hr') done
(412, 'dynamic') done
(413, 'tech') done
(414, 'qashqai') done
(415, 'nissana') done
(416, '2010r') done
(417, 'sam') done
(418, 'parkuje') done
(419, '216') done
(420, '115km') done
(421, 'active') done
(422, 'tourer') done
(423, 'ix55') done
(424, 'super') done
(425, 'auto') done
(426, 'rodzinne') done
(427, 'wakacje') done
(428, 'rav4') done
(429, 'jasna') done
(430, 'avant') done
(431, 'w') done
(432, 'bardzo') done
(433, 'dobrym') done
(434, 'stanie') done
(435, 'tronic') done
(436, '3008') done
(437, 'w212') done
(438, '2017') done
(439, 'canton') done
(440, '2xpdc') done
(441, 'el') done
(442, 'fotelepluspamięć') done
(443, '19"') done
(444, 'drzwi') done
(445, 'śliczna') done
(446, '85km') done
(447, 'ładny') done
(448, 'leszno') done
(449, 'volvo') done
(450, 'c30') done
(451, '6d') done
(452, '2007r') done
(453, 'polskory') done
(454, 'climatronic') done
(455, 'now

(810, '270') done
(811, 'style') done
(812, 'mały') done
(813, 'pilnie') done
(814, 'v50') done
(815, 'powypadkowy') done
(816, 'odpala') done
(817, 'wazne') done
(818, 'oc') done
(819, 'niezawodna') done
(820, 'd4d') done
(821, 'grande') done
(822, 'frontera') done
(823, 'a') done
(824, 'benzna') done
(825, '1996') done
(826, 'bezwypadkoywy') done
(827, 'l200') done
(828, '178km') done
(829, 'aut') done
(830, '8t') done
(831, 'agila') done
(832, '2lpg') done
(833, 'xsara') done
(834, 'citroena') done
(835, 'xsarę') done
(836, 'witam') done
(837, 'robione') done
(838, 'czas:') done
(839, '292000') done
(840, 'wymiana') done
(841, 'komplety') done
(842, 'regeneracja') done
(843, 'alternatora') done
(844, 'paska') done
(845, 'oraz') done
(846, 'wszystkich') done
(847, 'rolek') done
(848, '310000') done
(849, 'klocków') done
(850, 'hamulcowych') done
(851, 'tarcze') done
(852, 'przód') done
(853, 'płynu') done
(854, 'chłodzącego') done
(855, 'tysięcy') done
(856, 'temu') done
(857, 'wy') 

(1200, 'saxo') done
(1201, 'limitededition') done
(1202, 'niskiprzebieg') done
(1203, '4wd') done
(1204, 'lock') done
(1205, 'ładnystan') done
(1206, '2016r') done
(1207, 'dwa') done
(1208, 'f') done
(1209, '125km') done
(1210, 'własnego') done
(1211, 'benzplusgaz') done
(1212, 'xc90') done
(1213, 'orlando') done
(1214, 'antara') done
(1215, 'touch') done
(1216, 'connect') done
(1217, 'dużym') done
(1218, 'przeglądzie') done
(1219, 'ekskluzywna') done
(1220, 'skory') done
(1221, 'ecomotive') done
(1222, 'freemont') done
(1223, 'lounge') done
(1224, '6v6') done
(1225, 'wiek') done
(1226, '1997') done
(1227, 'touran') done
(1228, '105') done
(1229, '508') done
(1230, '102') done
(1231, 'm3') done
(1232, 'linea') done
(1233, 'zdjęcia') done
(1234, 'wkrótce') done
(1235, 'vi') done
(1236, 'tył') done
(1237, 'dotykowy') done
(1238, 'ekran') done
(1239, 'bmax') done
(1240, 'kluczyki') done
(1241, '2i') done
(1242, 'handsfree') done
(1243, 'sportback') done
(1244, '252km') done
(1245, 'latitu

(1575, '2hdi') done
(1576, 'caravelle') done
(1577, 'żaluzje') done
(1578, 'bezpośrednio') done
(1579, 'własciciel') done
(1580, 'w204') done
(1581, 'adam') done
(1582, '87km') done
(1583, 'glam') done
(1584, '116') done
(1585, 'spectra') done
(1586, 'impreza') done
(1587, 'b9') done
(1588, 'tribeca') done
(1589, 'problem') done
(1590, 'motorem') done
(1591, 'masarze') done
(1592, 'skretny') done
(1593, 'wypsazenie') done
(1594, '79100') done
(1595, 'e61') done
(1596, '525i') done
(1597, 'sprzedaż') done
(1598, 'motocykl') done
(1599, 'cbr') done
(1600, 'gsxr') done
(1601, '4motion') done
(1602, 'fsi') done
(1603, 'bvy') done
(1604, '180km') done
(1605, 'f25') done
(1606, 'matiz') done
(1607, 'friend') done
(1608, 'matiza') done
(1609, 'kpl') done
(1610, 'dodatkowych') done
(1611, 'airmaticx4') done
(1612, 'cof') done
(1613, 'pompowane') done
(1614, 'brc') done
(1615, 'pilot') done
(1616, 'bram') done
(1617, 'standard') done
(1618, 'w203') done
(1619, '34') done
(1620, 'converseplus') 

...

(3444, 'tytanium') done
(3445, 'xtr') done
(3446, 'hdi92km') done
(3447, 'sprawdż') done
(3448, 'ekstra') done
(3449, 'wnetrze') done
(3450, 'qashqaj') done
(3451, 'sprzedający') done
(3452, 'niekręcony') done
(3453, 'zar') done
(3454, '8500zł') done
(3455, 'croma') done
(3456, 'giugiaro') done
(3457, 'hedup') done
(3458, 'cuore') done
(3459, 'afn') done
(3460, 'ecoflex') done
(3461, '54tys') done
(3462, 'merc') done
(3463, 'pantera') done
(3464, 'bmwe46') done
(3465, 'volkswagena') done
(3466, '3c5') done
(3467, 'vsa') done
(3468, 'k20a3') done
(3469, 'ev1') done
(3470, 'właść') done
(3471, '66tkm') done
(3472, 'terracan') done
(3473, '9crdi') done
(3474, 'starszej') done
(3475, 'zimą') done
(3476, '6d4d') done
(3477, 'sti') done
(3478, 'avus') done
(3479, 'r17') done
(3480, '119') done
(3481, '2automat') done
(3482, '2014rok') done
(3483, '73tyś') done
(3484, '98km') done
(3485, 'sliki') done
(3486, 'doskonałym') done
(3487, 'a1') done
(3488, 'idealne') done
(3489, '50000') done
(349

(3808, '354km') done
(3809, '260kw') done
(3810, 'maja') done
(3811, 'v23%') done
(3812, 'freestyle') done
(3813, '2xkoła') done
(3814, 'wyposażone') done
(3815, 'opla"design') done
(3816, 'line"') done
(3817, 'years') done
(3818, 'tuning') done
(3819, 'boże') done
(3820, 'jaki') done
(3821, 'szybki') done
(3822, '300hp') done
(3823, 'c70') done
(3824, 'multipla') done
(3825, 'multi') done
(3826, 'tid') done
(3827, '160tyś') done
(3828, 'fiest') done
(3829, 'najbogatszej') done
(3830, '330xd') done
(3831, 'swiss') done
(3832, '1997r') done
(3833, 'niepowtarzalny') done
(3834, 'beats') done
(3835, '40tys') done
(3836, 'serwisowe') done
(3837, 'paket') done
(3838, 'benzin') done
(3839, 'bipper') done
(3840, '100%udokumentowany') done
(3841, '14r') done
(3842, 'strzał') done
(3843, 'dziesiątkę') done
(3844, 'linę') done
(3845, '328ix') done
(3846, '8lat') done
(3847, 'jednego') done
(3848, 'ictdi') done
(3849, 'licznik') done
(3850, '1100cm') done
(3851, 'fakt') done
(3852, '286') done
(3

(4172, 'szara') done
(4173, 'lne') done
(4174, 'cały') done
(4175, 'nawigacjaplus') done
(4176, 'skóryplus') done
(4177, 'ledplus') done
(4178, 'pieniądze') done
(4179, '55') done
(4180, 'tyskm') done
(4181, 'pryw') done
(4182, 'długa') done
(4183, 'skura') done
(4184, '118tyś') done
(4185, 'a5b5fl') done
(4186, '1właściel') done
(4187, 'opłayty') done
(4188, 'świetnie') done
(4189, 'odstąpię') done
(4190, 'leasing') done
(4191, 'dostępne') done
(4192, 'nr') done
(4193, 'lakierze') done
(4194, 'sto') done
(4195, 'procent') done
(4196, '44000tys') done
(4197, 'aso15') done
(4198, 'praktyczny') done
(4199, '115ps') done
(4200, '192') done
(4201, '472') done
(4202, 'm235i') done
(4203, '430i') done
(4204, '8pluslpg') done
(4205, 'rypin') done
(4206, 'vr6') done
(4207, 'l&k') done
(4208, '210kmplusgsiplus4x4plusfull') done
(4209, 'opcjaplusfaktura') done
(4210, 'valvenatic') done
(4211, 'mecedes') done
(4212, 'spaceback') done
(4213, 'tcr') done
(4214, '290km') done
(4215, 'additional') do

(4533, '52tkm') done
(4534, '6mt') done
(4535, '211000') done
(4536, '500x') done
(4537, 'multiaur') done
(4538, 'reanault') done
(4539, 'lauerate') done
(4540, 'fullserwis') done
(4541, '33000km') done
(4542, '8ts') done
(4543, 'eu6') done
(4544, 'connecta') done
(4545, '125ps') done
(4546, 'gwarancja24miesięcy') done
(4547, 'panora') done
(4548, 'pewne') done
(4549, 'brazowa') done
(4550, '145tys') done
(4551, '59tyś') done
(4552, 'podsufitówka') done
(4553, 'corollę') done
(4554, '0td4') done
(4555, 'etsi') done
(4556, 'mhev') done
(4557, '2020r') done
(4558, '184ps') done
(4559, 'ameryki') done
(4560, 'zainteresowania') done
(4561, '75kw') done
(4562, '2benz') done
(4563, '41') done
(4564, 'executiv') done
(4565, '210') done
(4566, 'octavię') done
(4567, 'is3') done
(4568, 'irmscher') done
(4569, '0multijet') done
(4570, 'koniecznie') done
(4571, '2ivtec') done
(4572, 'salonpl;2') done
(4573, '190km;navi;bi') done
(4574, "xenon;klimatronic;reling;pdc;alu17';vat") done
(4575, '134tk

(4897, 'atrakcyjna') done
(4898, 'niewielkich') done
(4899, 'dużoautzamiany') done
(4900, 'targówek') done
(4901, 'tysiak') done
(4902, 'dsg150') done
(4903, 'dsg190') done
(4904, 'shl170') done
(4905, 'x7') done
(4906, '138km') done
(4907, 'crdi141') done
(4908, 'd258') done
(4909, 'aluminiowych') done
(4910, 'ultra190') done
(4911, 'mocny') done
(4912, 'podgrzew') done
(4913, 'boosterjet') done
(4914, '111km') done
(4915, 'w218') done
(4916, 'sekwencyjny') done
(4917, 'zapraszm') done
(4918, 'autoryzowany') done
(4919, 'nawigacj') done
(4920, 'anglii') done
(4921, '326km') done
(4922, 'is200t') done
(4923, 'challenger') done
(4924, 'sxt') done
(4925, 'green') done
(4926, 'gwarancncja') done
(4927, 'v6benz') done
(4928, '2006rok') done
(4929, 'bjb') done
(4930, 'bolero') done
(4931, 'rifter') done
(4932, 'rlink') done
(4933, 'attraction') done
(4934, 'doiwestowany') done
(4935, '1168') done
(4936, 'gwarncja') done
(4937, 'klucz') done
(4938, 'cx9') done
(4939, '4m') done
(4940, 'multi

In [37]:
results

,index,variable,mean_1,mean_0,count_1,count_0,mean_diff
0,602,xd,94547.506130,31718.910695,571,55305,62828.595434
1,552,drive.,87644.526171,31633.201378,726,55150,56011.324793
2,347,drive,86687.980570,31599.845038,772,55104,55088.135531
3,849,xdrive|model,76194.921969,31697.593009,833,55043,44497.328960
4,582,xdrive|pakiet,73242.728261,31396.357221,1288,54588,41846.371040
...,...,...,...,...,...,...,...
1046,358,td,32265.387938,32370.883109,5256,50620,105.495172
1047,236,serwisowany,32452.955902,32358.467060,1474,54402,94.488842
1048,739,td.,32346.669728,32362.434077,5226,50650,15.764349
1049,756,us,32369.108140,32360.002780,5872,50004,9.105361


In [15]:
for part in enumerate(corpus):
    print(part)

(0, 'mazda')
(1, 'cx-3')
(2, '150km')
(3, '6at')
(4, '4x4')
(5, 'skypassion')
(6, 'plus')
(7, 'biala')
(8, 'skóra')
(9, 'safety')
(10, 'navi')
(11, 'peugeot')
(12, 'partner')
(13, 'opłacony~dokumentacja')
(14, 'przebiegu')
(15, 'kia')
(16, 'sportage')
(17, 'opłacona~napęd')
(18, 'škoda')
(19, 'roomster')
(20, 'opłacona~serwis')
(21, 'aso~klimatronic~grzane')
(22, 'siedzenia')
(23, 'opel')
(24, 'combo')
(25, 'klimatyzacja~przebieg~faktura~polski')
(26, 'salon')
(27, 'mitsubishi')
(28, 'lancer')
(29, '1.8')
(30, 'lancer,')
(31, 'polska,')
(32, '24000')
(33, 'przebiegu!')
(34, 'bmw')
(35, 'seria')
(36, '3')
(37, '320d')
(38, '5')
(39, 'e60')
(40, '535d')
(41, '272km')
(42, '2006r.')
(43, 'head')
(44, 'up')
(45, 'start')
(46, 'stop')
(47, 'xenony')
(48, 'bogate')
(49, 'wyposażenie')
(50, 'jeep')
(51, 'grand')
(52, 'cherokee')
(53, 'nan')
(54, '7')
(55, '740d')
(56, 'xdrive.')
(57, 'pl.')
(58, '1')
(59, 'właściciel.')
(60, 'soft')
(61, 'close.')
(62, 'komfortowy')
(63, 'dostęp.')
(64, 'volk

(2192, '#okazja,100%bezwypadkowy,serwisowany,1,4')
(2193, 'benz,stan')
(2194, 'idealny,#')
(2195, 'cheerokee')
(2196, 'prywatnej!')
(2197, 'a3/s3')
(2198, 'radionawigacja')
(2199, 'pewny')
(2200, 'sprzęgło,')
(2201, 'dwumasa,')
(2202, 'rozrząd,')
(2203, 'zarejestrowany!!!')
(2204, 'śląsk')
(2205, '157')
(2206, '1.6crdi')
(2207, 'oryg.lakier')
(2208, 'przeb')
(2209, '135000km')
(2210, 'fabryki!!')
(2211, '190km')
(2212, 'ss')
(2213, 'maptun')
(2214, 'performance')
(2215, 'traveller')
(2216, 'użytkowana')
(2217, 'jednej')
(2218, 'śliczny,')
(2219, '8j')
(2220, 'a8')
(2221, 'phaeton')
(2222, 'tdi,')
(2223, 'bezwypadkowa!')
(2224, 'ori')
(2225, 'lakier!')
(2226, 'sprzęgło!')
(2227, 'climatronic!')
(2228, 'zadbana!')
(2229, 'xc60')
(2230, 'f31')
(2231, 'podgrzewana')
(2232, 'kierownica')
(2233, 'el.bagażnik')
(2234, 'el.hak')
(2235, 'sportsize')
(2236, 'caravan')
(2237, '3.8')
(2238, 'jazdy!')
(2239, 'vat,')
(2240, 'camry')
(2241, 'godny')
(2242, 'polecenia')
(2243, 'gts')
(2244, 'nowa#gts#

(4212, 'benzynza')
(4213, '/150ps')
(4214, ';')
(4215, 'bezwypadku')
(4216, '183000')
(4217, 'centr')
(4218, 'zamek')
(4219, 'deluxe')
(4220, 'celerio')
(4221, 'produkcja')
(4222, '2019/')
(4223, 'wersja/')
(4224, 'silnik.')
(4225, 'odbudowany')
(4226, 'projekt,')
(4227, 'super,')
(4228, 'alufelgi16')
(4229, 'virtual')
(4230, 'oro')
(4231, 'alu16')
(4232, '2xparktronic')
(4233, '3xklimatronic')
(4234, 'fvat')
(4235, '328i')
(4236, 'sdrive18i')
(4237, 'bieg.')
(4238, '17",')
(4239, 'wyposazona')
(4240, 'sporty')
(4241, 'dpfu')
(4242, 'gazem,')
(4243, '164,5')
(4244, 'bluetooth/')
(4245, 'alcantara,')
(4246, 'turismo,')
(4247, 'topowa')
(4248, '2.0benz')
(4249, 'el.drzwi')
(4250, '2wd/automat/navi')
(4251, '/full')
(4252, 'led/kamera/gwarancja')
(4253, 'ogł.')
(4254, 'isofix/')
(4255, 'klimatyzaja')
(4256, 'eclipse')
(4257, 'fullled,')
(4258, 'tablet,')
(4259, 'cvt,')
(4260, 'logic')
(4261, '7,')
(4262, 'rolety')
(4263, 'katalizator')
(4264, 'irmicher')
(4265, 'sufit!!')
(4266, '1.6dci')

(6332, '18.12.2019')
(6333, '2590')
(6334, '209km')
(6335, 'alumy!')
(6336, '170km!z')
(6337, 'kamera!')
(6338, 'crdi!')
(6339, 'benzyna,wspomaganie,centralny,elek')
(6340, 'skóra!')
(6341, 'tempomat!')
(6342, '143000km!')
(6343, 'way')
(6344, 'twardy')
(6345, 'dysk')
(6346, 'tfsi/track')
(6347, 'style/dsg7/4moton/dcc/panorama/bixenon/hak/aso')
(6348, 'bezwypadkow')
(6349, 'zarejestr.')
(6350, 'volvo/')
(6351, '2xfiaty')
(6352, 'turbo!')
(6353, 'pl.tablice')
(6354, '120hp')
(6355, '5osob.')
(6356, 'titanium.')
(6357, 'ściągany')
(6358, '132tys')
(6359, 'gw')
(6360, '2001_')
(6361, 'asistance!')
(6362, '108tys')
(6363, 'bdb!')
(6364, 'vat23%!')
(6365, 'laureate!')
(6366, 'samochód!')
(6367, 'seria1')
(6368, 'd-mode')
(6369, 'ładna,')
(6370, '17’')
(6371, 'zadbany,serwisowany,')
(6372, 'luty')
(6373, 'edition1')
(6374, 'kupione')
(6375, 'volvo.pełna')
(6376, 'moż')
(6377, 'zam')
(6378, 'ferrari')
(6379, '458')
(6380, 'italia')
(6381, 'official')
(6382, 'dealer.')
(6383, '138000')
(6384, 

(8505, 'bora,')
(8506, 'strzała')
(8507, 'laurin&klement')
(8508, '1,2tsi')
(8509, 'dokumentacja!')
(8510, '/kuga')
(8511, 'czerwona')
(8512, 'nić')
(8513, 'busso')
(8514, '167tys.km.')
(8515, 'faceliftingu')
(8516, '71000')
(8517, 'puretech/allureplus/automat/salon')
(8518, 'pl/kamera')
(8519, '360/navi/el.')
(8520, '88km,')
(8521, 'ops')
(8522, '94tyś')
(8523, 'serwisie!')
(8524, 'drive,,')
(8525, '200t,')
(8526, 'enjoy,okazja')
(8527, '90km_100%')
(8528, 'oryginał_i')
(8529, 'wlaściciel_serwis')
(8530, 'aso_salon')
(8531, 'pl_')
(8532, '18000km\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f\u206f')
(8533, 'krajowy.pierwszy')
(8534, 'aso.100%oryginał.wart')
(8535, 'zakupu!')
(8536, 'jelenia')
(8537, 'góra')
(8538, 'i.')
(8539, 'sprzedaje')
(8540, 'prywatnie.')
(8541, 'uczciwie')
(8542, '740i')
(8543, 'trudne')
(8544, 'czasy')
(8545, '100%bezwyp.fv23%')
(8546, 'op')
(8547, 'kolorze')
(8548, 'białym')
(8549, 'mondeo.')
(8550, 'kombi.')
(8551, 

...

(15146, 'rozbite')
(15147, 'padnięty')
(15148, 'pl/serwisowany')
(15149, 'półskóra,alu,parktronic,okazja')
(15150, 'gaz/benzyna')
(15151, '1.8tpluslpg')
(15152, 'klimatronik,alu,tempomat')
(15153, '35900')
(15154, 'sound,')
(15155, 'funkcje')
(15156, 'jazdy,serw.')
(15157, 'niemiecn')
(15158, 'ładny,bezwypadkowy')
(15159, '3.1l')
(15160, 'jeep,')
(15161, ',1wł;')
(15162, '#1.')
(15163, '105km#bogate')
(15164, 'serwis#gwarancja')
(15165, 'ml350cdi')
(15166, 'individual/full')
(15167, 'rejes')
(15168, 'seriws')
(15169, 'całe')
(15170, '68km,')
(15171, '38069km')
(15172, '144tys.km')
(15173, 'wyjątkow')
(15174, 'radar,blis,ksenon')
(15175, ',skóra.')
(15176, 'wolksvagen')
(15177, 'passatse')
(15178, 'tdi105')
(15179, 'minivan.')
(15180, 'ładnne')
(15181, 'dualjet')
(15182, 'wersja,ledy,')
(15183, '2.0mjet')
(15184, 'irmscher,')
(15185, 'ele.klapa,')
(15186, '2004r.,')
(15187, 'radaracc')
(15188, 'websto')
(15189, '2kpl.kół.polecam')
(15190, 'enplusgaz')
(15191, '75km.')
(15192, '2.0cdi')


(18020, '85ps')
(18021, 'sprowadzony!')
(18022, 'i-tech')
(18023, '130km,led,navi,pdc')
(18024, 'przod')
(18025, 'tył,podg')
(18026, 'wignale')
(18027, 'lecie')
(18028, '203tys.')
(18029, '108500km')
(18030, 'nakładki')
(18031, 's-tronic,')
(18032, 'brygadowka')
(18033, '107m!!')
(18034, 'oszczędny.')
(18035, '3.0d/')
(18036, '23/')
(18037, 'opłatach!!!')
(18038, '87t.km!;klima;airbag;komputer;tempomat;abs;cd;el.szybyx4;śliczny!')
(18039, '2.0/142')
(18040, '115km.')
(18041, 'wielkim')
(18042, 'bólem')
(18043, 'serca')
(18044, 'cosmo1.4')
(18045, 'comfortplusnavi,salon')
(18046, 'eksploatacji!')
(18047, 'd4d!')
(18048, 'progressive')
(18049, '2007.130')
(18050, 'km,2,0')
(18051, 'serwisowa,opłacona')
(18052, 'zadbamy')
(18053, 'dostawczaka')
(18054, 'tysięcyprzebiegu')
(18055, 'sci')
(18056, '240kw')
(18057, 'x4,')
(18058, 'lust.,')
(18059, 'twojego')
(18060, 'salon,fv23%,i')
(18061, 'rej.2019,gwarancja,dealer')
(18062, 'szwajcar!')
(18063, 'wartości')
(18064, 'gdi-132')
(18065, 'km-co

(19874, 'prestigeplusled')
(19875, 'polska/bezwypadkowy/serwis')
(19876, 'aso/faktura!')
(19877, '28.')
(19878, 'widescreen')
(19879, '160.966km')
(19880, 'nawigacja,wspomaganie,')
(19881, 'area')
(19882, '460')
(19883, 'oferujemy')
(19884, 'pogrzewna')
(19885, ',1,6')
(19886, ',gwar,')
(19887, 'sunset')
(19888, 'parkow')
(19889, 'umowa')
(19890, 'kupna')
(19891, 'blues')
(19892, 'mulifunkcja')
(19893, 'maxi-dot')
(19894, 'ii.')
(19895, 'parkowani')
(19896, 'aż')
(19897, 'dom,')
(19898, 'multifunkcjia')
(19899, 'c180,1.6benzyna,navi,skóry,alu,stan')
(19900, 'super,mały')
(19901, 'przejęcia')
(19902, 'odświeżoną')
(19903, 'fabryczn')
(19904, 'siedzienia')
(19905, 'fr-line')
(19906, 'seatdriveprofil')
(19907, '"okazja"')
(19908, 'intelllink')
(19909, 'dw8c522')
(19910, 'bezyana')
(19911, 'nisana')
(19912, 'gold-x')
(19913, 'f30/f80')
(19914, 'active/fv23%/isofix/led/dealer/klim.automatyczna/krajowy')
(19915, 'drzwi,centralny')
(19916, 'zamek,kredyt,opłacony')
(19917, 'klimatyzacja,5')
(1

(22471, '4el.')
(22472, '89tys.km.')
(22473, 'nowa_145')
(22474, 'przebiegu_oryginalny')
(22475, 'lakier_1.6')
(22476, '8v_opłacona_ideał')
(22477, 'e220dt')
(22478, 'avantgarde/distronic/led/navi/kamera/alu19”/gwar.fabr/fv23%')
(22479, '72977km.')
(22480, 'przeb.119')
(22481, '2.0iplusgaz')
(22482, '#serwisowany')
(22483, 'cite')
(22484, 'brawo')
(22485, 'xenon\\')
(22486, 'drzwi\\')
(22487, '2014r,skóra,kamera,dvd,alu')
(22488, "17'',1")
(22489, 'wł,mały')
(22490, 'przebieg.stan')
(22491, 'bdb,bezwypadek')
(22492, 'f25/')
(22493, 'polskiegi')
(22494, 'salonu/')
(22495, 'scenic,')
(22496, 'orurowanie')
(22497, 'japan_stan')
(22498, "_keyless_alu16'")
(22499, 'soul,')
(22500, '83km,1')
(22501, 'wlascicel,oryg.73tys')
(22502, 'kmplusks.servisowa,klima')
(22503, 'automatyczna,cd')
(22504, 'merced')
(22505, 'klima,bardzo')
(22506, '87000km!!!')
(22507, 'highline,salon,serwis,alcantara,niski')
(22508, 'rail140km')
(22509, '28420')
(22510, 'propozycja!')
(22511, 'skup')
(22512, '311')
(2251

(24923, 'bixenon,solar,z')
(24924, 'niemec!!')
(24925, '1.6i~101ps~kolorowa')
(24926, 'nawigacja~pełen')
(24927, 'oryginał~unikatowy')
(24928, 'stan~top')
(24929, '6miejsc')
(24930, '2030')
(24931, "11'")
(24932, 'siedzących!!')
(24933, '2.00')
(24934, '108.000')
(24935, 'lusterka,bez')
(24936, 'wkładu,bezwypadkowy,serwis')
(24937, 'saolon')
(24938, '420ps')
(24939, 'os/klima/alu/tanio/pl')
(24940, '2.0gx')
(24941, 'sf')
(24942, 'xenon/full')
(24943, 'plusplus1.2')
(24944, '33000km')
(24945, 'topstan')
(24946, '166tyś.km.')
(24947, 'aktywne')
(24948, 'zawieszeni')
(24949, '1,6tdci,')
(24950, 'dwumasy,opłacony')
(24951, 'tkm!')
(24952, 'bezwypadkowa!full')
(24953, 'comfortplustechplusstyle')
(24954, 'bezwypadkowy/świeżo')
(24955, 'serwisie/faktura!')
(24956, 'kuga/escape')
(24957, '9800km')
(24958, 'tdcighia')
(24959, 'mpaciet')
(24960, 'lift/benzyna/oryginał/zadbany/opłacony')
(24961, 'niski,')
(24962, 'asystenci,')
(24963, 'kredyt/leasing')
(24964, 'miniak')
(24965, 'dł')
(24966, 'sty

In [9]:
pd.set_option('display.max_rows', 10000)

In [57]:
results.sort_values(by = 'mean_diff', ascending = False)

,index,variable,mean_1,mean_0,count_1,count_0,mean_diff
0,380,xd,95183.925553,31843.872633,497,47689,63340.052920
1,300,drive,87319.155120,31731.175750,664,47522,55587.979370
2,190,wd,71909.489899,32005.265927,594,47592,39904.223972
3,395,ale,69983.672199,32118.411936,482,47704,37865.260263
4,82,fv,67692.788476,31506.647855,1319,46867,36186.140622
5,199,pakiet,63046.694226,32006.310560,762,47424,31040.383666
6,611,akiet,62983.592398,32006.671278,763,47423,30976.921121
7,548,pak,62311.954603,31998.300867,793,47393,30313.653736
8,98,quattro,61711.662848,32121.976395,611,47575,29589.686453
9,434,sg,61518.505051,32012.199793,792,47394,29506.305257


In [51]:
corpus

['mazda',
 'cx',
 'km',
 'at',
 'x',
 'skypassion',
 'biala',
 'sk',
 'ra',
 'safety',
 'navi',
 'peugeot',
 'partner',
 'op',
 'acony',
 'dokumentacja',
 'przebiegu',
 'kia',
 'sportage',
 'acona',
 'nap',
 'd',
 'koda',
 'roomster',
 'serwis',
 'aso',
 'klimatronic',
 'grzane',
 'siedzenia',
 'opel',
 'combo',
 'klimatyzacja',
 'przebieg',
 'faktura',
 'polski',
 'salon',
 'mitsubishi',
 'lancer',
 'polska',
 'bmw',
 'seria',
 'e',
 'r',
 'head',
 'up',
 'start',
 'stop',
 'xenony',
 'bogate',
 'wyposa',
 'enie',
 'jeep',
 'grand',
 'cherokee',
 'nan',
 'xdrive',
 'pl',
 'w',
 'a',
 'ciciel',
 'soft',
 'close',
 'komfortowy',
 'dost',
 'p',
 'volkswagen',
 'polo',
 'lancia',
 'musa',
 'pi',
 'kny',
 'i',
 'oszcz',
 'dny',
 'honda',
 'accord',
 'vtec',
 'sport',
 'infiniti',
 'q',
 's',
 'uszkodzone',
 'prosto',
 'z',
 'ubezpieczalni',
 'cr',
 'v',
 'lpg',
 'stag',
 'mercedes',
 'benz',
 'clk',
 'automat',
 'avantgarde',
 'alu',
 'szyberdach',
 'xenon',
 'ford',
 'maverick',
 'civic',

In [ ]:
x.data.to_csv('data_with_corpus.csv')

In [73]:
cv = CountVectorizer(max_features = 331826)

In [74]:
cv.fit_transform(corpus).toarray()

MemoryError: 

In [18]:
x = x.add_dummies(['ford', 'mercedes', 'suzuki'], ['title', 'sub_title'])

In [19]:
x.data.head()

,ID,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable,title_ford,title_mercedes,title_suzuki,sub_title_ford,sub_title_mercedes,sub_title_suzuki
0,1,smart fortwo cabrio,18800,cabrio automat nawigacja kamera led szwajcaria,37000.0,2016,999,Benzyna,Bielsko-Biała,Śląskie,"Do negocjacji, Faktura VAT",0,0,0,0,0,0
1,2,ford kuga,34700,nawigacja/kamera cofania/zadbany/,198000.0,2011,2000,Diesel,Dąbrowa,Pomorskie,Do negocjacji,1,0,0,0,0,0
2,3,chevrolet captiva 2.4,27900,2.4 136km klima alus navi dvd skóra 4x4 poleca...,246000.0,2009,2405,Benzyna,Płock,Mazowieckie,Do negocjacji,0,0,0,0,0,0
3,4,renault clio 1.2,37900,1.2 benz 73 km !! idealny !! full !! warszawa,15000.0,2017,1149,Benzyna,Warszawa,Mazowieckie,Do negocjacji,0,0,0,0,0,0
4,5,opel meriva,7900,mały przebieg 109 tyś/km bezwypadkowy,109000.0,2006,1400,Benzyna,Żyrardów,Mazowieckie,NaN,0,0,0,0,0,0


In [48]:
data = x.data

In [58]:
data2 = AddDummy(data, 'title', 'ford')

In [66]:
data['price'].sum()

786919459

In [3]:
x = CarData('data/')

In [4]:
x.outliers()

Offers with price greater than 200000 have been discarded
The number of such offers = 294

Offers with mileage greater than 400000 have been discarded
The number of such offers = 166

Offers with year lower than 1995 have been discarded
The number of such offers = 288

Offers with engine_cm3 greater than 750 and lower than 4000 have been discarded
The number of such offers = 378

Total number of discarded offers = 1126(4.68%) - may be different to the sum of above due to overlap


In [5]:
x.data.head()

,ID,title,price,sub_title,mileage_km,year,engine_cm3,engine_type,city,province,negotiable
0,1,Smart Fortwo cabrio,18800,Cabrio Automat Nawigacja Kamera Led Szwajcaria,37000.0,2016,999,Benzyna,Bielsko-Biała,Śląskie,"Do negocjacji, Faktura VAT"
1,2,Ford Kuga,34700,Nawigacja/kamera cofania/Zadbany/,198000.0,2011,2000,Diesel,Dąbrowa,Pomorskie,Do negocjacji
2,3,Chevrolet Captiva 2.4,27900,2.4 136km klima alus navi DVD skóra 4x4 Poleca...,246000.0,2009,2405,Benzyna,Płock,Mazowieckie,Do negocjacji
3,4,Renault Clio 1.2,37900,1.2 BENZ 73 KM !! IDEALNY !! FULL !! Warszawa,15000.0,2017,1149,Benzyna,Warszawa,Mazowieckie,Do negocjacji
4,5,Opel Meriva,7900,mały przebieg 109 tyś/km bezwypadkowy,109000.0,2006,1400,Benzyna,Żyrardów,Mazowieckie,NaN


In [20]:
x.missing, x.duplicates

(737, 8782)

In [64]:
x.describe()

,price,mileage_km,year,engine_cm3
count,24073.00,24073.00,24073.00,24073.00
mean,32688.88,159511.92,2009.88,1789.87
std,31457.86,83724.08,5.33,483.63
min,250.00,0.00,1995.00,796.00
25%,12200.00,99000.00,2006.00,1461.00
50%,22900.00,163000.00,2010.00,1794.00
75%,41800.00,217000.00,2014.00,1997.00
max,200000.00,400000.00,2020.00,4000.00


In [ ]:
x.scatter_nox()

In [ ]:
plt.hist(x.data['mileage_km'], bins = 100)
plt.title('abc')
plt.show()

plt.hist(x.data['price'],bins = 100)
plt.show()

plt.hist(x.data['year'], bins = 100)
plt.show()

plt.hist(x.data['engine_cm3'], bins = 100)
plt.show()

In [ ]:
x.data[x.data['price']>1100000]

In [ ]:
x.data[x.data['mileage_km']>1500000]

In [ ]:
x.data[x.data['year']<1990]

In [ ]:
round(pd.DataFrame(
    data = x.data[['price', 'mileage_km', 'year', 'engine_cm3']].describe(),
    columns = x.data[['price', 'mileage_km', 'year', 'engine_cm3']].columns),2)

In [ ]:
x.data.isnull().sum()

In [ ]:
data = pd.read_csv('run_results_21-04-2020.csv')

In [ ]:
data = ReadAndPreprocess(df)

In [ ]:
df = pd.read_csv('data/run_results_21-04-2020.csv')
df.head()

In [ ]:
df2 = pd.read_csv('data/run_results_25-04_2020.csv')
df2.head()

In [ ]:
df3 = pd.read_csv('data/run_results_26-04-2020.csv')
df3.head()

In [ ]:
string = re.sub('[^a-zA-Z]', ' ', self.data.reset_index().loc[i]['concat_title_subtitle'])
string = string.split()

In [17]:
string = 'Ala MA KOtA123łćż            '

In [18]:
string_sub = re.sub('[^a-zA-Z]', ' ', string)
string_sub

'Ala MA KOtA                  '

In [20]:
string_split = string_sub.split()
string_split

['Ala', 'MA', 'KOtA']